In [1]:
import json
import sys
import os
import itertools

ifn = "./ch09_new.transformers.ipynb" # sys.argv[1]
ofn = os.path.join("../../manuscript/includes/", os.path.basename(ifn).rsplit('.', 1)[0]+'.inc')   # sys.argv[2]
with open(ifn) as nb_file:
    nb = json.load(nb_file)

def count_leading_spaces(s):
    non_space = [i for i, c in enumerate(s) if not c.isspace()]
    if non_space:
        return non_space[0]
    else:
        return 99

def format_cell(cell_tag, code, out_file,  output_list=None):
    cell_tag = cell_tag.replace('adoc-', '', 1)
    print("// tag::{}[]".format(cell_tag), file=out_file)
    print("// tag::{}_code[]".format(cell_tag), file=out_file)
    print("[source,python]\n----", file=out_file)
    # print("# In[{}]:".format(cell_tag), file=out_file)
    book_lines = [l for l in code if "# bookskip" not in l]
    common_blanks = min(count_leading_spaces(l) for l in book_lines)
    if common_blanks > 0:
        book_lines = [l[common_blanks:] for l in book_lines]
    
    callouts = []
    for i, l in enumerate(book_lines):
        if '# callout:' in l:
            code, callout = l.split("# callout:", 1)
            callouts.append(callout.strip())
            code = f"{code}# <{len(callouts)}>\n"
            book_lines[i] = code

    for i, l in enumerate(book_lines):
        if len(l) > 75:
            print(f"cell {cell_tag} input line {i} has {len(l)} characters:", l)

    print("".join(book_lines), file=out_file)
    print('----', file=out_file)
    for i, c in enumerate(callouts, start=1):
        print(f"<{i}> {c}", file=out_file)
    print("// end::{}_code[]".format(cell_tag), file=out_file)

    if output_list:
        print("", file=out_file)
        print("// tag::{}_output[]".format(cell_tag), file=out_file)
        print("----", file=out_file)

        print("// Out[{}]:".format(cell_tag), file=out_file)
        for i, l in enumerate(output_list):
            if len(l) > 75:
                print(
                    f"cell {cell_tag} output line {i} has {len(l)} characters:",
                    l,
                )
        print("".join(output_list), file=out_file)
        print("----", file=out_file)
        print("// end::{}_output[]".format(cell_tag), file=out_file)
    print("// end::{}[]".format(cell_tag), file=out_file)
    print("\n", file=out_file)
    
with open(ofn, "w", encoding="utf8") as out_file:
    for cell_index, cell_dict in enumerate(
        c for c in nb["cells"] if c["cell_type"] == "code"
    ):
        tags = cell_dict['metadata'].get('tags', []) + cell_dict.get("tags", [])
        if any(t.startswith('adoc-') for t in tags):
            for cell_tag in tags:
                code = cell_dict["source"]
                
                output_list = []
                for output_dict in cell_dict["outputs"]:
                    if output_list:
                        output_list += ["\n"]
                    output_list += output_dict.get("text", [])
                    output_list += output_dict.get("data", {}).get("text/plain", [])
                
                format_cell(cell_tag, code, out_file,  output_list=output_list)
        source = cell_dict["source"]
        source_tags = [l.replace("# tag::", "", 1).rstrip() for l in source if l.lstrip().startswith("# tag::")]
        for t in source_tags:
            part_source = []
            in_tag = False
            for l in source:
                if l.lstrip().rstrip() == "# end::" + t:
                    in_tag = False
                if in_tag:
                    part_source.append(l)
                if l.lstrip().rstrip() == "# tag::" + t:
                    in_tag = True
            if in_tag:
                print(f"tag {tag} never closed!")
            format_cell(t, part_source, out_file,  output_list=[])
            

cell patchembed input line 17 has 78 characters:         x = torch.cat([self.class_token.expand(bs, -1, -1), x], dim=1)  # <4>

